In [ ]:
import os
import glob
import torch
from torch import nn
import numpy as np

import albumentations
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
from tensorflow import keras

import config
import dataset
import engine
import train
from model import CaptchaModel

In [7]:

image_files = glob.glob(os.path.join(config.DEPLOYMENT_DATA, "*.png"))
"""The glob module finds all the pathnames matching a specified pattern according
to the rules used by the Unix shell, although results are returned in arbitrary order.
No tilde expansion is done, but *, ?, and character ranges expressed with []
will be correctly matched. This is done by using the os.scandir() and fnmatch.fnmatch()
functions in concert, and not by actually invoking a subshell."""
targets_orig = [x.split("/")[-1][9:14] for x in image_files]
targets = [[c for c in x] for x in targets_orig]
targets_flat = [c for clist in targets for c in clist]
lbl_enc = preprocessing.LabelEncoder()
lbl_enc.fit(targets_flat)
#print(len(lbl_enc.classes_))
targets_enc = [lbl_enc.transform(x) for x in targets]
targets_enc = np.array(targets_enc)
targets_enc = targets_enc + 1
(
    train_imgs,
    test_imgs,
    train_targets,
    test_targets,
    _,
    test_targets_orig,
) = model_selection.train_test_split(
    image_files, targets_enc, targets_orig, test_size=0.1, random_state=42
)

train_dataset = dataset.ClassificationDataset(
    image_paths=train_imgs,
    targets=train_targets,
    resize=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH),
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    shuffle=True,
)
test_dataset = dataset.ClassificationDataset(
    image_paths=test_imgs,
    targets=test_targets,
    resize=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH),
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    shuffle=False,
)
print("lbl_enc.classes_ ",len(lbl_enc.classes_))
model = CaptchaModel(num_chars=len(lbl_enc.classes_))
model.load_state_dict(torch.load(f"model_50.pt"))
model.eval()

model.to(config.DEVICE)
predictions, lossdata = engine.eval_fn(model, )
print(predictions)
print("this is lost data ",lossdata)



lbl_enc.classes_  28


RuntimeError: Error(s) in loading state_dict for CaptchaModel:
	size mismatch for output.weight: copying a param with shape torch.Size([37, 64]) from checkpoint, the shape in current model is torch.Size([29, 64]).
	size mismatch for output.bias: copying a param with shape torch.Size([37]) from checkpoint, the shape in current model is torch.Size([29]).